# Accessing DC2 data in PostgreSQL at NERSC

This notebook demonstrates use of the PostgreSQL database at NERSC.  Currently the only available dataset is the object catalog for Run1.1p.

### Prerequisites
* A file ~/.pgpass containing a line like this:

`nerscdb03.nersc.gov:54432:desc_dc2_drp:desc_dc2_drp_user:`_password_

Ask me what the value for _password_ should be.  This line allows you to use the desc_dc2_drp_user account, which has *SELECT* privileges on the database, without entering a password in plain text. There is a separate account for adding to or modifying the database. .pgpass must be protected so that only owner may read and write it.

* Access to the psycopg2 package which provides a Python interface to PostgreSQL. If it's not already available, do a local pip install.  Then you may have to 
  * put a line like this in your .bashrc.ext:  
    ```$DESCPYTHONPATH=~/.local/pythonpath3.6/site-packages```
  * restart your jupyter-dev server
  
This notebook uses psycopg2 directly for queries.  It is also possible to use sqlalchemy but you will still need a PostgreSQL driver. Of these psycopg2 is the most popular.


In [5]:
import psycopg2

import numpy as np
import pandas

Make the db connection

In [6]:
dbname = 'desc_dc2_drp'
dbuser = 'desc_dc2_drp_user'
dbhost = 'nerscdb03.nersc.gov'
schema = 'run11p'
dbconfig = {'dbname' : dbname, 'user' : dbuser, 'host' : dbhost}
dbconn = psycopg2.connect(**dbconfig)

Tables for the Run1.1p data are in the *schema* (acts like a namespace) run11p. There is a special system schema, **information_schema**, which contains tables describing the structure of user tables. Of these **information_schema.columns** is most likely to be useful. The following lists all tables and views belonging to schema run11p. (I will use the convention of writing SQL keywords in all caps in queries. It's not necessary; the SQL interpreter ignores case.)

In [7]:
q1 = "SELECT DISTINCT table_name FROM information_schema.columns WHERE table_schema='{schema}' ORDER BY table_name".format(**locals())
with dbconn.cursor() as cursor:
    cursor.execute(q1)
    for record in cursor:
        print(record[0])

forced
forced2
forced3
_forced:part1
_forced:part2
_forced:part3
_forced:position
meas
meas2
meas3
meas4
_meas:part1
_meas:part2
_meas:part3
_meas:part4
_meas:position


The items with simple names, like **forced** or **meas2** are views; those starting with an underscore are the underlying tables.  The typical user will query only the views. Information is broken across several tables or views because there are too many columns for a single table. All tables and views have a field ```object_id``` which is convenient for joining when you don't know which view has the particular quantity you care about. Here is a list of all quantities in the view **forced**. It contains the most basic quantities for forced photometry.

In [37]:
tbl = 'forced'
q2 = "SELECT column_name, data_type FROM information_schema.columns WHERE table_schema='{schema}' AND table_name='{tbl}'".format(**locals())
with dbconn.cursor() as cursor:
    cursor.execute(q2)
    records = cursor.fetchall()
    print("There are {} columns in table {}.  They are:\n".format(len(records), tbl))
    print("Name                                                     Data Type")
    for record in records:
        print("{0!s:55}  {1!s:20}".format(record[0], record[1]) )

There are 567 columns in table forced.  They are:

Name                                                     Data Type
object_id                                                bigint              
ra                                                       double precision    
dec                                                      double precision    
skymap_id                                                integer             
tract                                                    integer             
patch                                                    integer             
patch_s                                                  text                
parent_id                                                bigint              
nchild                                                   integer             
isprimary                                                boolean             
detect_ispatchinner                                      boolean             
detect_istractinner     

Here is a query which counts up objects per tract.  

In [14]:
q3 = "SELECT tract, COUNT(object_id) from {schema}.forced WHERE isprimary GROUP BY tract".format(**locals())
with dbconn.cursor() as cursor:
    cursor.execute(q3)
    df = pandas.DataFrame(cursor.fetchall(), columns=['tract', 'count'])
    print(df)
 

    tract   count
0    4430  103887
1    4431  155478
2    4432  137634
3    4433      11
4    4636    1446
5    4637  344628
6    4638  664922
7    4639  681528
8    4640  299132
9    4848  332054
10   4849  785795
11   4850  719228
12   4851  754903
13   4852   10778
14   5062   13640
15   5063  538497
16   5064  569393
17   5065  579765
18   5066  199661
